<a href="https://colab.research.google.com/github/Hugo-Zh0/YoloV12-Object-Detection-Project/blob/main/YOLOv12_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# 🚀 YOLOv12 Object Detection Project

[![Python](https://img.shields.io/badge/Python-3.10%2B-blue.svg)](https://www.python.org/downloads/)
[![Anaconda](https://img.shields.io/badge/Anaconda-Navigator-green.svg)](https://www.anaconda.com/download)
[![VS Code](https://img.shields.io/badge/Editor-VS%20Code-blue.svg)](https://code.visualstudio.com/)
[![Ultralytics](https://img.shields.io/badge/YOLOv12-Ultralytics-yellow.svg)](https://github.com/ultralytics/ultralytics)
[![License](https://img.shields.io/badge/License-MIT-lightgrey.svg)](LICENSE)

---
<br>

## 📌 Overview
A **collaborative group project** by Swinburne University students in partnership with **CSIRO**.  
This repository contains the setup, configuration, and workflow for training and running **YOLOv12** object detection models.

**👨‍💻 Team Members:** Harron, Feng, Bunmi, Huss, Hugo.

---
**Repo:** `Hugo-Zh0/YoloV12-Object-Detection-Project`  
**What you’ll do:**
1. Check runtime & GPU
2. Install dependencies
3. Clone your repo
4. Set paths in repository
5. Train
6. Validate
7. Predict
8. Export
10. Troubleshoot


## Step 1 — 🚀 Runtime & GPU check

In [1]:

#@title Check Python, CUDA, and PyTorch
import sys, platform
print("Python:", sys.version)
print("Platform:", platform.platform())
try:
    import torch
    print("PyTorch:", torch.__version__)
    print("CUDA available:", torch.cuda.is_available())
    if torch.cuda.is_available():
        print("CUDA device:", torch.cuda.get_device_name(0))
except Exception as e:
    print("PyTorch not installed yet (will install in next step).")


Python: 3.12.11 (main, Jun  4 2025, 08:56:18) [GCC 11.4.0]
Platform: Linux-6.1.123+-x86_64-with-glibc2.35
PyTorch: 2.8.0+cu126
CUDA available: True
CUDA device: Tesla T4


## Step 2 — ⬇️ Install dependencies

In [2]:

#@title Install Ultralytics & helpers
!pip install ultralytics

import torch, cv2, ultralytics
print("Ultralytics:", ultralytics.__version__)
print("Torch:", torch.__version__, "| CUDA:", torch.cuda.is_available())
print("OpenCV:", cv2.__version__)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 27.3 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics: 8.3.186
Torch: 2.8.0+cu126 | CUDA: True
OpenCV: 4.12.0


## Step 3 —  🤖 Clone your repository

In [ ]:

#@title Clone repo
REPO_URL = "https://github.com/Hugo-Zh0/YoloV12-Object-Detection-Project"
REPO_DIR = "/content/YoloV12-Object-Detection-Project"

import shutil, os, pathlib
if os.path.isdir(REPO_DIR):
    shutil.rmtree(REPO_DIR)

!git clone -q {REPO_URL} {REPO_DIR}
print("Cloned into:", REPO_DIR)


Then Next: Create 2 Folder called **yaml** and **datasets** inside the project repository

In [4]:

#@title Unzip Dataset and save to dataset folder
!unzip -q /content/Koala.zip -d /content/YoloV12-Object-Detection-Project/datasets/koala

Move YAML File to YAML Folder

## Step 4 — 🎞️ Set model & data paths

Change YAML File for correct location
change to /content/YoloV12-Object-Detection-Project/xxx

## Step 5 — 💥 Train (set your parameters)

In [ ]:
from ultralytics import YOLO
import torch
import os

# --- Direct paths (edit to your repo mount point in Colab) ---
MODEL_PATH = "/content/YoloV12-Object-Detection-Project/models/yolov12s.pt"
DATA_YAML  = "/content/YoloV12-Object-Detection-Project/yaml/data.yaml"

print("Model path:", MODEL_PATH)
print("Data yaml :", DATA_YAML)

# Device selection
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

# Load model + train
model = YOLO(MODEL_PATH)
results = model.train(
    data=DATA_YAML,
    epochs=100,
    imgsz=960,
    batch=12,
    workers=2,   # Colab usually fine with >1
    device=device,
)

print("Save dir:", getattr(results, "save_dir", "/content/runs/train"))


## Step 6 — ✅ Validate best model

In [ ]:

#@title Validate best.pt
from pathlib import Path
from ultralytics import YOLO

RUNS_DIR = Path("/content/runs/train")
assert RUNS_DIR.exists(), "No training runs found. Train first."

latest = sorted(RUNS_DIR.glob("**/"), key=lambda p: p.stat().st_mtime)[-1]
best = latest / "weights" / "best.pt"
print("Using:", best)
assert best.exists(), f"Missing best.pt in {latest}/weights"

model = YOLO(str(best))
metrics = model.val(data=str(DATA_YAML), imgsz=960, device="cuda" if __import__("torch").cuda.is_available() else "cpu")
print(metrics)


## Step 7 — 🔮 Predict (images)

In [ ]:

#@title Upload images and predict
from google.colab import files
from ultralytics import YOLO
from pathlib import Path
import torch

uploaded = files.upload()
img_paths = list(uploaded.keys())
print("Uploaded:", img_paths)

# choose model: latest best or repo weights
RUNS_DIR = Path("/content/runs/train")
best_path = None
if RUNS_DIR.exists():
    cands = sorted(RUNS_DIR.glob("**/weights/best.pt"), key=lambda p: p.stat().st_mtime)
    if cands:
        best_path = str(cands[-1])
model_path = best_path if best_path else str(Path("/content/YoloV12-Object-Detection-Project") / "models" / "yolov12s.pt")

model = YOLO(model_path)
pred = model.predict(source=img_paths, imgsz=960, device="cuda" if torch.cuda.is_available() else "cpu", save=True)
print("Saved to:", pred[0].save_dir if isinstance(pred, list) and pred else "Check /content/runs/predict")


## Step 8 — 📺 Predict (video)

In [ ]:

#@title Predict on a video (URL or upload)
video_url = ""  #@param {type:"string"}

import urllib.request, os
from google.colab import files
from ultralytics import YOLO
from pathlib import Path
import torch

src = None
if video_url.strip():
    src = "/content/input_video.mp4"
    try:
        urllib.request.urlretrieve(video_url, src)
        print("Downloaded:", src)
    except Exception as e:
        print("Failed to download:", e)
        src = None

if not src:
    up = files.upload()
    if up:
        src = list(up.keys())[0]
        print("Uploaded:", src)

assert src, "No video provided."

RUNS_DIR = Path("/content/runs/train")
best_path = None
if RUNS_DIR.exists():
    cands = sorted(RUNS_DIR.glob("**/weights/best.pt"), key=lambda p: p.stat().st_mtime)
    if cands:
        best_path = str(cands[-1])
model_path = best_path if best_path else str(Path("/content/YoloV12-Object-Detection-Project") / "models" / "yolov12s.pt")

model = YOLO(model_path)
pred = model.predict(source=src, imgsz=960, device="cuda" if torch.cuda.is_available() else "cpu", save=True)
print("Video predictions saved to:", pred[0].save_dir if isinstance(pred, list) and pred else "Check /content/runs/predict")


## Step 9 — ♟️ Exporting

In [ ]:
## Nothing here yet

## Troubleshooting


- **Weights YAML missing:** ensure `models/yolov12s.pt` and `yaml/data.yaml` exist in the repo or update paths.
- **No GPU:** Colab may not offer a GPU; the notebook will use CPU (much slower).
- **Val fails:** Train first; then rerun the validate cell.
- **Poor metrics:** Add more data, correct labels, tune `imgsz`/`batch`/`epochs`.
